In [15]:
import pandas as pd
from custom_func import *

In [29]:
quartal_1 = "https://data.gov.ua/dataset/95c82130-4310-4edb-b7e5-f01b59385eb0/resource/caa8e037-3020-492d-9ca0-7db4429a0025/revision/166988/download"
latest = "https://data.gov.ua/dataset/95c82130-4310-4edb-b7e5-f01b59385eb0/resource/caa8e037-3020-492d-9ca0-7db4429a0025/download/all_declarations_pmd.csv"
dtype = {"legal_entity_edrpou": str}

df_P06_007_data = pd.read_csv(latest, dtype=dtype)
df_P06_007_1kv_data = pd.read_csv(quartal_1, dtype=dtype)
df_P06_007 = df_P06_007_data.copy()
df_P06_007_1kv = df_P06_007_1kv_data.copy()

In [57]:
def change(current, previous):
    """Оскільки датасет зберігає версійність, можемо стежити за динамікою. 
    Це не входить в параметри, але цікаво. """
    
    g1 = current.groupby("division_area", as_index=False)["count_declarations"].sum() \
                .rename(columns={"count_declarations": "Поточний квартал"})
    
    g2 = previous.groupby("division_area", as_index=False)["count_declarations"].sum() \
                .rename(columns={"count_declarations": "Попередній квартал"})
    
    res = pd.merge(g2,g1)
    res["Зміна"] = res["Поточний квартал"] - res["Попередній квартал"]
    res = res.rename(columns={"division_area": "region"})
    res["region"] = res["region"].str.title()
    
    return res.loc[res["region"].ne("М.Київ")].copy()

In [58]:
res = change(current=df_P06_007, previous=df_P06_007_1kv)

In [59]:
res

,region,Попередній квартал,Поточний квартал,Зміна
0,Івано-Франківська,1103637,1114319,10682
1,Вінницька,1309642,1318755,9113
2,Волинська,858940,863025,4085
3,Дніпропетровська,2484484,2501702,17218
4,Донецька,1358544,1367677,9133
5,Житомирська,993624,996194,2570
6,Закарпатська,979716,998379,18663
7,Запорізька,1318436,1328142,9706
8,Кіровоградська,692996,699007,6011
9,Київська,1502997,1516491,13494


---

In [3]:
df_P06_007["legal_entity_edrpou"] = df_P06_007["legal_entity_edrpou"].str.zfill(8)
df_P06_007 = df_P06_007.loc[df_P06_007["division_area"].ne("М.КИЇВ")].copy()

In [4]:
df_P06_007_fin = df_P06_007.groupby("division_area", as_index=False)["count_declarations"].sum() \
                           .rename(columns={"division_area": "region"})

In [11]:
df_P06_007_fin["region"] = df_P06_007_fin["region"].str.title()
df_P06_007_fin["population"] = df_P06_007_fin["region"].map(dict_population)
df_P06_007_fin['p6_01_raw'] = df_P06_007_fin["count_declarations"]/df_P06_007_fin["population"]

In [13]:
normalize_parameter(df_P06_007_fin, "p6_01_raw", "p6_01",min_bound=0,max_bound=1)

Емпіричні границі: (0, 1)
Коефіцієнти нормалізації: (1.0, -0.0)
Параметр p6_01 нормалізовано і додано до таблиці



In [14]:
df_P06_007_fin

,region,count_declarations,population,p6_01_raw,p6_01
0,Івано-Франківська,1114319,1366161,0.815657,0.815657
1,Вінницька,1318755,1541782,0.855345,0.855345
2,Волинська,863025,1029736,0.838103,0.838103
3,Дніпропетровська,2501702,3180809,0.786499,0.786499
4,Донецька,1367677,4127532,0.331355,0.331355
5,Житомирська,996194,1212157,0.821836,0.821836
6,Закарпатська,998379,1251467,0.797767,0.797767
7,Запорізька,1328142,1691424,0.785221,0.785221
8,Кіровоградська,699007,930013,0.751610,0.751610
9,Київська,1516491,1771229,0.856180,0.856180
